In [ ]:
import requests
import json
import pandas as pd
import seaborn as sns; 

### Set our list of targets, the disease to run against and number of top associations to filter through

In [ ]:
mytargets = ['DRD2', 'DRD4', 'PINK1', 'GRIN1', 'FUS', 'ANG', 'FRED', 'BERT']
disease = "EFO_0002508"     # Parkinson's
#disease = "EFO_0000253"     # ALS
maxresults = 100             # max 10,000

### Make the API call with our disease to find the top results (ordered by overall score descending)

In [ ]:
url = 'https://www.targetvalidation.org/api/latest/public/association/filter'
headers = {"Accept": "application/json"}
data = "{\"disease\":[\"" + disease + "\"], \"size\":" + str(maxresults) + "}"

response = requests.post(url, headers=headers, data=data)
output = response.json()

### Lets see what we got back

In [ ]:
print json.dumps(output, indent=2)

### Run through all the targets and identify those we're interested in, adding them to a Pandas DataFrame

In [ ]:
df = pd.DataFrame()

for target in output['data']:
    
    # If the target is in our list then add it to the DataFrame
    if target['target']['gene_info']['symbol'] in mytargets:

        # Add all the association scores in one hit
        s = pd.Series(target['association_score']['datatypes'])
        # Append the overall score
        s['overall'] = target['association_score']['overall']
        # Use the target as the series name
        s.name = str(target['target']['gene_info']['symbol'])
        # Append the series to our DataFrame
        df = df.append(s, ignore_index=False)

        
# Reorder and rename columns to match OpenTargets viz
df = df.reindex(columns=['overall','genetic_association','somatic_mutation','known_drug','affected_pathway','rna_expression','literature','animal_model'])
df.columns = ['Association score', 'Genetic associations', 'Somatic mutations', 'Drugs', 'Affected pathways', 'RNA expression', 'Text mining', 'Animal models']

### Now we have just the data we're interested in, lets see which of our targets have associations in OpenTargets and which don't

In [ ]:
print "Targets that have an association to " + disease + ":"
print df.index.tolist()
print
print "Targets that had no recorded association in the top " + str(maxresults) + " results returned for disease " + disease + ":"
print list(set(mytargets)-set(df.index))

### Lets print the list of matching targets and their scores

In [ ]:
print df

### Plot this DataFrame as a simple heatmap using the Seaborn vizualization library

In [ ]:
%matplotlib inline
sns.set()
ax = sns.heatmap(df, cmap="Blues", annot=True, linewidths=2)
sns.plt.show()